1. Basic run ChatGPT => ghi kết quả vào file sheet hiện tại

Làm FullStack nên có 1 bước khá quan trọng: 
1. Prompting ban đầu chỉ cần test có work hay không => Prompting sau đó design các keys như nào: Detect thì cần keys gì, gợi ý chấm thì cần keys gì, ... 
2. Back-end: Điều chỉnh CHUNG 1 LOẠI JSON từ Prompting để Back-end dễ lấy là đẹp nhất, hoặc ko Back-end sẽ cần tinh chỉnh code nhiều với mỗi loại JSON khác nhau. Chẳng hạn: 


Cách 1: 
```


{
  "teaching_vocab_pron_grammar": [
    {
      "start": "<minute:second of start>",
      "end": "<minute:second of end>",
      "tag":"<one or more tags: vocab, pronun, grammar>", 
      "extracted_transcriptions": "<Extracted vocabulary, pronunciation and grammar>",
      "explain": "<explain for detection>"
    },
    ...
  ]
}

```

Cách 2: 

```


{
  "teaching_vocab_pron_grammar1": {
      "start": "<minute:second of start>",
      "end": "<minute:second of end>",
      "tag":"<one or more tags: vocab, pronun, grammar>", 
      "extracted_transcriptions": "<Extracted vocabulary, pronunciation and grammar>",
      "explain": "<explain for detection>"
    },
  "teaching_vocab_pron_grammar2": {
      "start": "<minute:second of start>",
      "end": "<minute:second of end>",
      "tag":"<one or more tags: vocab, pronun, grammar>", 
      "extracted_transcriptions": "<Extracted vocabulary, pronunciation and grammar>",
      "explain": "<explain for detection>"
    },
    ...
    
}

```

Cách 2 sẽ gần với cách lấy của ... => Chỉ cần design 1 hàm chung thôi
```
{
  "warm_up": {
    "start": "<minute:second of start>",
    "end": "<minute:second of end>", 
    "extracted_transcription": "<>",
    "explain": "<explain for detection>"
  },
  "lead-in": {
    "start": "<minute:second of start>",
    "end": "<minute:second of end>", 
    "extracted_transcription": "<>",
    "explain": "<explain for detection>"
  },
  "wrap-up": {
    "start": "<minute:second of start>",
    "end": "<minute:second of end>", 
    "extracted_transcription": "<>",
    "explain": "<explain for detection>"
  }
}
```

3. UI: trong lúc code UI, tuỳ vào UI hiển thị mà có thể điều chỉnh JSON hoặc TEXT (chẳng hạn lên Web App thì là JSON, còn App Scripts có khi chỉ cần để Text)

## Initial version: 

### 1. Build 3 hàm `getOpenAIResponse(systemPrompt, userInputPrompt)` để lấy systemPrompt và userInputPrompt từ file `getDataFromSheet.js` => `writeAssistantOutput(sheetName, range, result)` để viết kết quả vào file

Nhật ký phát triển: 
- Initially, I intended to define the getOpenAIResponse function with three parameters: systemPrompt, userPrompt, and outputRangeSheetFile, ... but later changed it as shown above, 

- Ban đầu tôi có 4 hàm: getData, get rompt từ Data, getOpenAIResponse(systemPrompt, userInputPrompt), and writeResponse,
Tuy nhiên hàm getOpenAIResponse() bản chất cũng chỉ nhận 2 lần getData => Bỏ hàm getPrompt đi hợp lý

Output thu được 3 hàm: 

1. `getDataFromSheet(sheetName, cellAddress)`
   - Retrieves data from a specific cell in a Google Sheet.
   - Parameters: sheet name and cell address (e.g., "A1").
   - Returns the cell value.

2. `getOpenAIResponse(systemPrompt, userInputPrompt, apiKey)`
   - Sends a request to the OpenAI API and returns the response.
   - Parameters: system prompt, user input prompt, and API key.
   - Returns the AI model's response as a string.

3. `writeResponseToSheet(sheetName, range, result)`
   - Writes the OpenAI response to a specific range in a Google Sheet.
   - Parameters: sheet name, cell range (e.g., "A1"), and the response to write.

These functions allow you to read data from a sheet, process it through OpenAI's API, and write the results back to the sheet.

Note: Ensure you have the necessary permissions and API key to use these functions.

### 2. Ý tưởng ban đầu Need to write a function (WINDOW EXTEND CONTEXT): 
- Parameters 1: Full transcriptions of 1 video, variable name: ...
- Parameters 2: start time of the video segment, 
- Parameters 3: end time of the video segment,
- Parameters 4: desired time extension on both sides, default is 2 minutes. 
--------
response: The transcriptions have been extended on both sides by the specified time.

===========



Giới thiệu ngắn gọn
- Mối liên hệ hàm chính hàm phụ
    - Chức năng của từng hàm
    - Tham số gì, return gì?


#### Hệ thống mở rộng đoạn transcription video

Hàm chính: `functExtendTranscriptionSegment`

Chức năng: Mở rộng một đoạn cụ thể của transcription video.

Tham số:
- transcriptionStr: Chuỗi transcription đầy đủ
- sections: Đối tượng chứa thông tin các phần của video
- key: Tên phần cần mở rộng
- extensionTime: Thời gian mở rộng (giây)

Trả về: Chuỗi transcription đã mở rộng

#### Hàm phụ trợ:

1. `getStartEndTimes`
   - Chức năng: Lấy thời gian bắt đầu và kết thúc của một phần
   - Tham số: sections, key
   - Trả về: Đối tượng {start, end}

2. `parseTranscription`
   - Chức năng: Chuyển đổi chuỗi transcription thành mảng đối tượng
   - Tham số: transcriptionStr
   - Trả về: Mảng các đối tượng transcription

3. `convertTimeToSeconds`
   - Chức năng: Chuyển đổi chuỗi thời gian thành số giây
   - Tham số: timeString
   - Trả về: Số giây

Các hàm phụ trợ này hỗ trợ hàm chính trong việc xử lý dữ liệu đầu vào và thực hiện các phép tính cần thiết để mở rộng đoạn transcription.

### 3. Cân nhắc về việc sử dụng đối tượng cấu hình (CONFIG):

-  Tạo `CONFIG` sớm giúp tổ chức mã tốt hơn, dễ mở rộng và tránh sử dụng giá trị cố định (hard-coded). 
- Tuy nhiên, nếu chưa rõ số lượng tham số cần cấu hình, bạn có thể bắt đầu với `CONFIG` nhỏ và linh hoạt, hoặc `SỬ DỤNG BIẾN TOÀN CỤC THAY VÌ CỤC BỘ` tạm thời và ghi chú trong mã để dễ dàng cập nhật sau này.

Quan trọng là:

- **Duy trì tính nhất quán** trong cách đặt tên biến và tổ chức mã để dễ dàng chuyển đổi sang `CONFIG` sau này.
- **Sẵn sàng cho việc refactor** khi dự án mở rộng.
- **Lên kế hoạch cho tương lai** bằng cách theo dõi các tham số cần cấu hình và dành thời gian để tổ chức lại mã khi cần thiết.


```
      EXTENDED_TRANSCRIPTION: {
        WARM_UP: 'C2',
        LEAD_IN: 'C3',
        WRAP_UP: 'C4',
        // Ô trắng: C5.
        TEACHING_VOCAB_PRONUN_GRAMMAR_1: 'C6',
        TEACHING_VOCAB_PRONUN_GRAMMAR_2: 'C7',
        TEACHING_VOCAB_PRONUN_GRAMMAR_3: 'C8',
        TEACHING_VOCAB_PRONUN_GRAMMAR_4: 'C9',
        TEACHING_VOCAB_PRONUN_GRAMMAR_5: 'C10',
        TEACHING_VOCAB_PRONUN_GRAMMAR_6: 'C11',
        // Ô trắng: C12.
        TEACHING_ICQS_CCQS_1: 'C12',
        TEACHING_ICQS_CCQS_2: 'C13',
        TEACHING_ICQS_CCQS_3: 'C14',
        TEACHING_ICQS_CCQS_4: 'C15',
        TEACHING_ICQS_CCQS_5: 'C16',
      },

      // EXTENDED_TRANSCRIPTION ở cột C, thì sẽ SUG_SCORING ở cột D cùng hàng
      get SUG_SCORING() {
        const extendedTranscription = this.EXTENDED_TRANSCRIPTION;
        const sugScoring = {};
        for (const [key, value] of Object.entries(extendedTranscription)) {
          const [col, row] = value.split('');
          sugScoring[key] = String.fromCharCode(col.charCodeAt(0) + 1) + row;
        }
        return sugScoring;
      }
```

Giấu API đi


1. Set up

Từ
```
const CONFIG = {
  API_KEY: '...'
}
```
Thành
```

const CONFIG = {
  API_KEY: PropertiesService.getScriptProperties().getProperty('OPENAI_API_KEY'),
  // ... rest of the code
}
```


c. In the Apps Script editor, go to Settings (gear icon) > Script Properties.
d. Click "Add script property".
e. Set the property name as 'OPENAI_API_KEY' and the value as their API key.
f. Click "Save script properties".
2. Provide instructions:
When sharing the sheet, include these instructions for setting up the API key.

3. Optional: Create a user interface for setting the API key:
You can create a custom menu item that opens a dialog for users to input their API key. Here's an example of how to do this:
config.js
```
// ... existing CONFIG object

function onOpen() {
  SpreadsheetApp.getUi()
    .createMenu('API Settings')
    .addItem('Set API Key', 'showApiKeyDialog')
    .addToUi();
}

function showApiKeyDialog() {
  var ui = SpreadsheetApp.getUi();
  var result = ui.prompt(
    'Enter API Key',
    'Please enter your OpenAI API key:',
    ui.ButtonSet.OK_CANCEL
  );

  if (result.getSelectedButton() == ui.Button.OK) {
    PropertiesService.getScriptProperties().setProperty('OPENAI_API_KEY', result.getResponseText());
    ui.alert('API Key set successfully');
  }
}
```


### 4. CONFIG, ĐÓNG FULL LUỒNG, TỐI ƯU CÁC HÀM VIẾT HÀM CLEAN, 

Bug: 
```
Kiểm tra xem, tại sao: 
- Khi tôi chạy hàm: function runSugScoringAll()

Thì :2 hàm này chạy được ```runsugScoringWarmLeadWrap, runSugScoringVocabPronunGrammarAll chạy được MÀ HÀM:   runSugScoringICQsCCQsAll(); LẠI KO CHẠY ĐƯỢC """

Tuy nhiên khi chạy riêng hàm runSugScoringICQsCCQsAll thì lại chạy được, hoặc đảo hàm này lên đầu thì chạy được

```

Có thể có một số nguyên nhân tiềm ẩn gây ra vấn đề này:
- Thời gian chạy quá lâu: Nếu hai hàm đầu tiên mất nhiều thời gian để chạy, có thể hàm thứ ba không được thực thi do giới hạn thời gian của Google Apps Script.
- Lỗi trong hai hàm đầu: Nếu có lỗi xảy ra trong runSugScoringWarmLeadWrap() hoặc runSugScoringVocabPronunGrammarAll(), quá trình thực thi có thể bị dừng trước khi đến runSugScoringICQsCCQsAll().
- Giới hạn quota: Nếu hai hàm đầu tiên sử dụng hết quota của Google Apps Script, hàm thứ ba có thể không được thực thi.
- HOẶC LÀ: BUG BÁO, NHƯNG VẪN SẼ CHẠY TIẾP CHỨ KHÔNG BỊ DỪNG